<a href="https://colab.research.google.com/github/davidlealo/100profes/blob/master/multiple_models_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uso de múltiples modelos de AI



#### Paso 1: Instalación de librerías

In [1]:
!pip install gradio requests -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00


#### Paso 2: Configuración inicial

In [2]:
import os
import requests

# Variable global para almacenar la API key del usuario
user_api_key = None

#### Paso 3: Función para interactuar con Mistral


In [3]:
def mistral_request(prompt, api_key):
    """
    Envía un prompt a la API de Mistral y devuelve la respuesta.
    """
    url = "https://api.mistral.ai/v1/completions"  # Cambia si usas otro endpoint
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mistral",  # O el modelo que uses
        "prompt": prompt,
        "max_tokens": 100
    }
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        return response.json().get("choices")[0].get("text", "No response.")
    else:
        return f"Error: {response.status_code} - {response.text}"

#### Paso 4: Identificar intención y asignar acción


In [4]:
def identify_and_call_model(prompt, api_key):
    """
    Detecta la intención del usuario y realiza la acción correspondiente.
    """
    if "análisis de sentimientos" in prompt.lower():
        return mistral_request("Realiza un análisis de sentimientos: " + prompt, api_key)
    elif "traducción" in prompt.lower():
        return mistral_request("Traduce lo siguiente: " + prompt, api_key)
    else:
        return mistral_request(prompt, api_key)


#### Paso 5: Interfaz con Gradio

In [6]:
# Paso 5: Interfaz con Gradio (Separar configuración y chat)

# Interfaz para configurar la API Key
def api_key_interface(api_key):
    """
    Guarda la API Key del usuario.
    """
    global user_api_key
    user_api_key = api_key
    return "API Key configurada correctamente."

# Interfaz para el chat
def chat_interface(user_input):
    """
    Procesa el prompt del usuario y devuelve la respuesta del modelo.
    """
    if not user_api_key:
        return "Por favor, configura tu API Key primero."
    try:
        response = identify_and_call_model(user_input, user_api_key)
        return f"Respuesta de la AI:\n{response}"
    except Exception as e:
        return f"Error procesando tu solicitud: {str(e)}"

# Crear dos interfaces independientes
api_key_ui = gr.Interface(
    fn=api_key_interface,
    inputs=gr.Textbox(label="Introduce tu API Key", type="password", placeholder="Tu API Key aquí"),
    outputs=gr.Textbox(label="Estado de configuración"),
    title="Configurar API Key",
    description="Introduce tu API Key de Mistral para comenzar."
)

chat_ui = gr.Interface(
    fn=chat_interface,
    inputs=gr.Textbox(label="Prompt", placeholder="Escribe tu solicitud aquí..."),
    outputs=gr.Textbox(label="Respuesta", lines=10),
    title="Chat con Mistral",
    description="Envía un prompt y obtén una respuesta del modelo."
)

# Combinar las interfaces en un espacio de trabajo
gr.TabbedInterface(
    [api_key_ui, chat_ui],
    ["Configurar API Key", "Chat"]
).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://31c63d481e64d78b1b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Nuevo enfoque

In [8]:
# Paso 1: Instalación de librerías necesarias
!pip install gradio transformers -q

# Paso 2: Importación y configuración
import os
from transformers import pipeline

# Modelos predefinidos con tareas compatibles
models = {
    "text-to-image": "CompVis/stable-diffusion-v1-4",  # Imagen desde texto
    "text-to-video": "runwayml/stable-diffusion-v1-5",  # Generación de video
    "text-to-audio": "suno/bark",  # Generación de audio
    "text-generation": "gpt2"  # Generación de texto
}

# Función para cargar pipelines dinámicamente según el modelo
pipelines = {}

def load_model(task):
    """
    Carga el pipeline de Hugging Face según la tarea.
    """
    if task not in pipelines:
        pipelines[task] = pipeline(task=task, model=models[task])
    return pipelines[task]

# Paso 3: Detección de intención
def detect_task(prompt):
    """
    Detecta el tipo de tarea basado en palabras clave en el prompt.
    """
    if any(keyword in prompt.lower() for keyword in ["imagen", "dibujo", "gráfico"]):
        return "text-to-image"
    elif any(keyword in prompt.lower() for keyword in ["video", "animación"]):
        return "text-to-video"
    elif any(keyword in prompt.lower() for keyword in ["audio", "música", "sonido"]):
        return "text-to-audio"
    else:
        return "text-generation"  # Tarea por defecto: generación de texto

# Paso 4: Procesar solicitud según el modelo
def process_prompt(prompt):
    """
    Detecta la intención del usuario, carga el modelo correspondiente y genera un resultado.
    """
    task = detect_task(prompt)

    # Cargar el pipeline adecuado
    model_pipeline = load_model(task)

    # Generar salida
    if task == "text-to-image":
        # La salida puede ser una imagen (se ajusta según el pipeline)
        return "Imagen generada exitosamente (implementa visualización)."
    elif task == "text-to-video":
        return "Video generado exitosamente (implementa visualización)."
    elif task == "text-to-audio":
        return "Audio generado exitosamente (implementa visualización)."
    else:
        # Para tareas de texto
        result = model_pipeline(prompt, max_length=100, num_return_sequences=1)
        return f"Task: {task}\n\nOutput:\n{result[0]['generated_text']}"

# Paso 5: Interfaz con Gradio
import gradio as gr

# Interfaz única para procesar prompts
def main_function(user_input):
    """
    Procesa la entrada del usuario y devuelve el resultado del modelo adecuado.
    """
    try:
        response = process_prompt(user_input)
        return response
    except Exception as e:
        return f"Error procesando tu solicitud: {str(e)}"

interface = gr.Interface(
    fn=main_function,
    inputs=gr.Textbox(label="Prompt", placeholder="Describe lo que necesitas (imagen, video, audio, texto...)"),
    outputs=gr.Textbox(label="Resultado", lines=10),
    title="Chat Multi-Modo con Modelos Específicos",
    description="Un chatbot que detecta automáticamente si necesitas texto, imagen, video o audio."
)

# Ejecutar la interfaz
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ff9e625e23bd2f0317.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Enfoque imagenes

In [1]:
# Paso 1: Instalación de librerías necesarias
!pip install gradio transformers diffusers accelerate safetensors -q

# Paso 2: Importación y configuración
import torch
from transformers import pipeline
from diffusers import StableDiffusionPipeline

# Modelos específicos según la tarea
models = {
    "text-to-image": "runwayml/stable-diffusion-v1-5",  # Texto a imagen
    "text-generation": "gpt2",  # Generación de texto
}

# Paso 3: Funciones para cada tarea
def generate_image(prompt):
    """
    Genera una imagen basada en el texto usando un modelo de Diffusers.
    """
    model_id = models["text-to-image"]
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe = pipe.to("cuda")  # Asegúrate de tener una GPU
    image = pipe(prompt).images[0]
    return image

def generate_text(prompt):
    """
    Genera texto basado en un prompt usando un modelo de Transformers.
    """
    model_id = models["text-generation"]
    text_generator = pipeline("text-generation", model=model_id)
    output = text_generator(prompt, max_length=100, num_return_sequences=1)
    return output[0]["generated_text"]

# Paso 4: Detección de tarea
def detect_task(prompt):
    """
    Detecta la tarea en base al contenido del prompt.
    """
    if any(keyword in prompt.lower() for keyword in ["imagen", "dibujo", "gráfico"]):
        return "text-to-image"
    else:
        return "text-generation"  # Tarea por defecto

# Paso 5: Procesar el prompt
def process_prompt(prompt):
    """
    Procesa el prompt y devuelve la salida según la tarea detectada.
    """
    task = detect_task(prompt)
    if task == "text-to-image":
        return generate_image(prompt)
    elif task == "text-generation":
        return generate_text(prompt)

# Paso 6: Interfaz con Gradio
import gradio as gr

def main_function(prompt):
    """
    Detecta la tarea y devuelve el resultado en la interfaz.
    """
    task = detect_task(prompt)
    try:
        if task == "text-to-image":
            return process_prompt(prompt)
        elif task == "text-generation":
            return process_prompt(prompt)
    except Exception as e:
        return f"Error procesando tu solicitud: {str(e)}"

# Creación de interfaces específicas
text_output = gr.Textbox(label="Resultado (Texto)")
image_output = gr.Image(label="Resultado (Imagen)")

# Interfaz principal con salidas dinámicas
interface = gr.Interface(
    fn=main_function,
    inputs=gr.Textbox(label="Prompt", placeholder="Describe lo que necesitas: texto o imagen"),
    outputs=[image_output, text_output],
    title="Chat Multi-Modo",
    description="Detecta automáticamente si necesitas texto o imágenes y usa el modelo adecuado."
)

# Lanzar la interfaz
interface.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.1 MB/s eta 0:00:00
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://381e78b615d9646dde.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spa

# otro enfoque

In [4]:
# Paso 1: Instalación de librerías necesarias
!pip install gradio transformers diffusers accelerate safetensors -q

# Paso 2: Importación y configuración
import torch
from transformers import pipeline
from diffusers import StableDiffusionPipeline
import gradio as gr
import gc

# Modelos específicos según la tarea
models = {
    "text-to-image": "runwayml/stable-diffusion-v1-5",  # Texto a imagen
    "text-generation": "gpt2",  # Generación de texto
}

# Paso 3: Funciones para cada tarea
def generate_image(prompt):
    """
    Genera una imagen basada en el texto usando Diffusers.
    """
    try:
        # Cargar el modelo
        model_id = models["text-to-image"]
        pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
        pipe = pipe.to("cuda")  # Asegúrate de usar GPU

        # Generar imagen
        image = pipe(prompt).images[0]
        return image
    except Exception as e:
        print(f"Error generando la imagen: {str(e)}")
        return f"Error generando la imagen: {str(e)}"

def generate_text(prompt):
    """
    Genera texto basado en un prompt usando un modelo de Transformers.
    """
    try:
        # Cargar el modelo
        model_id = models["text-generation"]
        text_generator = pipeline("text-generation", model=model_id)

        # Generar texto
        output = text_generator(prompt, max_length=100, num_return_sequences=1)
        return output[0]["generated_text"]
    except Exception as e:
        print(f"Error generando el texto: {str(e)}")
        return f"Error generando el texto: {str(e)}"

# Paso 4: Detección de tarea
def detect_task(prompt):
    """
    Detecta la tarea en base al contenido del prompt.
    """
    if any(keyword in prompt.lower() for keyword in ["imagen", "dibujo", "gráfico"]):
        return "text-to-image"
    else:
        return "text-generation"  # Tarea por defecto

# Paso 5: Procesar el prompt
def process_prompt(prompt):
    """
    Procesa el prompt y devuelve la salida según la tarea detectada.
    """
    task = detect_task(prompt)

    # Liberar memoria antes de cargar modelos
    torch.cuda.empty_cache()
    gc.collect()

    if task == "text-to-image":
        return generate_image(prompt)
    elif task == "text-generation":
        return generate_text(prompt)

# Paso 6: Interfaz con Gradio
def main_function(prompt):
    """
    Detecta la tarea y devuelve el resultado en la interfaz.
    """
    task = detect_task(prompt)
    try:
        if task == "text-to-image":
            return process_prompt(prompt)
        elif task == "text-generation":
            return process_prompt(prompt)
    except Exception as e:
        print(f"Error procesando tu solicitud: {str(e)}")
        return f"Error procesando tu solicitud: {str(e)}"

# Configuración de la interfaz
interface = gr.Interface(
    fn=main_function,
    inputs=gr.Textbox(label="Prompt", placeholder="Describe lo que necesitas: texto o imagen"),
    outputs=[
        gr.Image(label="Resultado (Imagen)", type="pil"),
        gr.Textbox(label="Resultado (Texto)")
    ],
    title="Chat Multi-Modo",
    description="Detecta automáticamente si necesitas texto o imágenes y usa el modelo adecuado."
)

# Lanzar la interfaz
interface.launch(server_name="0.0.0.0")


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://544bf5eca5e834cf72.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# enfoque

In [5]:
# Paso 1: Instalación de librerías necesarias
!pip install gradio transformers diffusers accelerate safetensors -q

# Paso 2: Importación y configuración
import torch
from transformers import pipeline
from diffusers import StableDiffusionPipeline
import gradio as gr
import gc

# Modelos específicos según la tarea
models = {
    "text-to-image": "runwayml/stable-diffusion-v1-5",  # Texto a imagen
    "text-generation": "gpt2",  # Generación de texto
}

# Paso 3: Funciones para cada tarea
def generate_image(prompt):
    """
    Genera una imagen basada en el texto usando Diffusers.
    """
    try:
        # Cargar el modelo
        model_id = models["text-to-image"]
        pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
        pipe = pipe.to("cuda")  # Asegúrate de usar GPU

        # Generar imagen
        image = pipe(prompt).images[0]
        return image
    except Exception as e:
        print(f"Error generando la imagen: {str(e)}")
        return f"Error generando la imagen: {str(e)}"

def generate_text(prompt):
    """
    Genera texto basado en un prompt usando un modelo de Transformers.
    """
    try:
        # Cargar el modelo
        model_id = models["text-generation"]
        text_generator = pipeline("text-generation", model=model_id)

        # Generar texto
        output = text_generator(prompt, max_length=100, num_return_sequences=1)
        return output[0]["generated_text"]
    except Exception as e:
        print(f"Error generando el texto: {str(e)}")
        return f"Error generando el texto: {str(e)}"

# Paso 4: Detección de tarea
def detect_task(prompt):
    """
    Detecta la tarea en base al contenido del prompt.
    """
    if any(keyword in prompt.lower() for keyword in ["imagen", "dibujo", "gráfico"]):
        return "text-to-image"
    else:
        return "text-generation"  # Tarea por defecto

# Paso 5: Procesar el prompt
def process_prompt(prompt):
    """
    Procesa el prompt y devuelve la salida según la tarea detectada.
    """
    task = detect_task(prompt)

    # Liberar memoria antes de cargar modelos
    torch.cuda.empty_cache()
    gc.collect()

    if task == "text-to-image":
        return generate_image(prompt)
    elif task == "text-generation":
        return generate_text(prompt)

# Paso 6: Interfaz con Gradio
def main_function(prompt):
    """
    Detecta la tarea y devuelve el resultado en la interfaz.
    """
    task = detect_task(prompt)
    try:
        if task == "text-to-image":
            return process_prompt(prompt)
        elif task == "text-generation":
            return process_prompt(prompt)
    except Exception as e:
        print(f"Error procesando tu solicitud: {str(e)}")
        return f"Error procesando tu solicitud: {str(e)}"

# Configuración de la interfaz
interface = gr.Interface(
    fn=main_function,
    inputs=gr.Textbox(label="Prompt", placeholder="Describe lo que necesitas: texto o imagen"),
    outputs=[
        gr.Image(label="Resultado (Imagen)", type="pil"),
        gr.Textbox(label="Resultado (Texto)")
    ],
    title="Chat Multi-Modo",
    description="Detecta automáticamente si necesitas texto o imágenes y usa el modelo adecuado."
)

# Lanzar la interfaz
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://620e2dd5abee23315a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# enfoque

In [6]:
# Paso 1: Instalación de librerías necesarias
!pip install gradio transformers diffusers accelerate safetensors -q

# Paso 2: Importación y configuración
import torch
from transformers import pipeline
from diffusers import StableDiffusionPipeline
import gradio as gr
import gc

# Modelos específicos según la tarea
models = {
    "text-to-image": "runwayml/stable-diffusion-v1-5",  # Texto a imagen
    "text-generation": "gpt2",  # Generación de texto
}

# Paso 3: Funciones para cada tarea
def generate_image(prompt):
    """
    Genera una imagen basada en el texto usando Diffusers.
    """
    try:
        # Verificar disponibilidad de GPU
        if not torch.cuda.is_available():
            return "Error: GPU no disponible. Asegúrate de usar un entorno con GPU."

        # Cargar el modelo
        model_id = models["text-to-image"]
        print("Cargando modelo de Diffusers para text-to-image...")
        pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
        pipe = pipe.to("cuda")

        # Generar imagen
        print("Generando imagen...")
        image = pipe(prompt).images[0]
        return image
    except Exception as e:
        print(f"Error generando la imagen: {str(e)}")
        return f"Error generando la imagen: {str(e)}"

def generate_text(prompt):
    """
    Genera texto basado en un prompt usando un modelo de Transformers.
    """
    try:
        # Cargar el modelo
        model_id = models["text-generation"]
        print("Cargando modelo de Transformers para text-generation...")
        text_generator = pipeline("text-generation", model=model_id)

        # Generar texto
        print("Generando texto...")
        output = text_generator(prompt, max_length=100, num_return_sequences=1)
        return output[0]["generated_text"]
    except Exception as e:
        print(f"Error generando el texto: {str(e)}")
        return f"Error generando el texto: {str(e)}"

# Paso 4: Detección de tarea
def detect_task(prompt):
    """
    Detecta la tarea en base al contenido del prompt.
    """
    if any(keyword in prompt.lower() for keyword in ["imagen", "dibujo", "gráfico"]):
        return "text-to-image"
    else:
        return "text-generation"  # Tarea por defecto

# Paso 5: Procesar el prompt
def process_prompt(prompt):
    """
    Procesa el prompt y devuelve la salida según la tarea detectada.
    """
    task = detect_task(prompt)

    # Liberar memoria antes de cargar modelos
    torch.cuda.empty_cache()
    gc.collect()

    if task == "text-to-image":
        return generate_image(prompt)
    elif task == "text-generation":
        return generate_text(prompt)

# Paso 6: Interfaz con Gradio
def main_function(prompt):
    """
    Detecta la tarea y devuelve el resultado en la interfaz.
    """
    task = detect_task(prompt)
    try:
        print(f"Detectando tarea: {task}")
        if task == "text-to-image":
            return process_prompt(prompt), None
        elif task == "text-generation":
            return None, process_prompt(prompt)
    except Exception as e:
        print(f"Error procesando tu solicitud: {str(e)}")
        return None, f"Error procesando tu solicitud: {str(e)}"

# Configuración de la interfaz
interface = gr.Interface(
    fn=main_function,
    inputs=gr.Textbox(label="Prompt", placeholder="Describe lo que necesitas: texto o imagen"),
    outputs=[
        gr.Image(label="Resultado (Imagen)", type="pil"),
        gr.Textbox(label="Resultado (Texto)")
    ],
    title="Chat Multi-Modo",
    description="Detecta automáticamente si necesitas texto o imágenes y usa el modelo adecuado."
)

# Lanzar la interfaz
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://52b162941962124446.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# enfoque

In [7]:
# Paso 1: Instalación de librerías necesarias
!pip install gradio transformers diffusers accelerate safetensors -q

# Paso 2: Importación y configuración
import torch
from transformers import pipeline
from diffusers import StableDiffusionPipeline
import gradio as gr
import gc

# Modelos específicos según la tarea
models = {
    "text-to-image": "runwayml/stable-diffusion-v1-5",  # Texto a imagen
    "text-generation": "gpt2",  # Generación de texto
}

# Paso 3: Funciones para cada tarea
def generate_image(prompt):
    """
    Genera una imagen basada en el texto usando Diffusers.
    """
    try:
        # Verificar disponibilidad de GPU
        if torch.cuda.is_available():
            device = "cuda"
            torch_dtype = torch.float16
            print("Usando GPU para la generación de imagen.")
        else:
            device = "cpu"
            torch_dtype = torch.float32
            print("Advertencia: GPU no disponible. Usando CPU, lo cual puede ser más lento.")

        # Cargar el modelo
        model_id = models["text-to-image"]
        print("Cargando modelo de Diffusers para text-to-image...")
        pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch_dtype)
        pipe = pipe.to(device)

        # Generar imagen
        print("Generando imagen...")
        image = pipe(prompt).images[0]
        return image
    except Exception as e:
        print(f"Error generando la imagen: {str(e)}")
        return f"Error generando la imagen: {str(e)}"

def generate_text(prompt):
    """
    Genera texto basado en un prompt usando un modelo de Transformers.
    """
    try:
        # Cargar el modelo
        model_id = models["text-generation"]
        print("Cargando modelo de Transformers para text-generation...")
        text_generator = pipeline("text-generation", model=model_id)

        # Generar texto
        print("Generando texto...")
        output = text_generator(prompt, max_length=100, num_return_sequences=1)
        return output[0]["generated_text"]
    except Exception as e:
        print(f"Error generando el texto: {str(e)}")
        return f"Error generando el texto: {str(e)}"

# Paso 4: Detección de tarea
def detect_task(prompt):
    """
    Detecta la tarea en base al contenido del prompt.
    """
    if any(keyword in prompt.lower() for keyword in ["imagen", "dibujo", "gráfico"]):
        return "text-to-image"
    else:
        return "text-generation"  # Tarea por defecto

# Paso 5: Procesar el prompt
def process_prompt(prompt):
    """
    Procesa el prompt y devuelve la salida según la tarea detectada.
    """
    task = detect_task(prompt)

    # Liberar memoria antes de cargar modelos
    torch.cuda.empty_cache()
    gc.collect()

    if task == "text-to-image":
        return generate_image(prompt)
    elif task == "text-generation":
        return generate_text(prompt)

# Paso 6: Interfaz con Gradio
def main_function(prompt):
    """
    Detecta la tarea y devuelve el resultado en la interfaz.
    """
    task = detect_task(prompt)
    try:
        print(f"Detectando tarea: {task}")
        if task == "text-to-image":
            return process_prompt(prompt), None
        elif task == "text-generation":
            return None, process_prompt(prompt)
    except Exception as e:
        print(f"Error procesando tu solicitud: {str(e)}")
        return None, f"Error procesando tu solicitud: {str(e)}"

# Configuración de la interfaz
interface = gr.Interface(
    fn=main_function,
    inputs=gr.Textbox(label="Prompt", placeholder="Describe lo que necesitas: texto o imagen"),
    outputs=[
        gr.Image(label="Resultado (Imagen)", type="pil"),
        gr.Textbox(label="Resultado (Texto)")
    ],
    title="Chat Multi-Modo",
    description="Detecta automáticamente si necesitas texto o imágenes y usa el modelo adecuado."
)

# Lanzar la interfaz
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7198e3a31fe9e63115.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# enfoque async

In [8]:
# Paso 1: Instalación de librerías necesarias
!pip install gradio transformers diffusers accelerate safetensors -q

# Paso 2: Importación y configuración
import torch
from transformers import pipeline
from diffusers import StableDiffusionPipeline
import gradio as gr
import gc
import asyncio

# Modelos específicos según la tarea
models = {
    "text-to-image": "runwayml/stable-diffusion-v1-5",  # Texto a imagen
    "text-generation": "gpt2",  # Generación de texto
}

# Paso 3: Funciones para cada tarea
async def generate_image(prompt):
    """
    Genera una imagen basada en el texto usando Diffusers.
    """
    try:
        # Verificar disponibilidad de GPU
        if torch.cuda.is_available():
            device = "cuda"
            torch_dtype = torch.float16
            print("Usando GPU para la generación de imagen.")
        else:
            device = "cpu"
            torch_dtype = torch.float32
            print("Advertencia: GPU no disponible. Usando CPU, lo cual puede ser más lento.")

        # Cargar el modelo
        model_id = models["text-to-image"]
        print("Cargando modelo de Diffusers para text-to-image...")
        pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch_dtype)
        pipe = pipe.to(device)

        # Generar imagen
        print("Generando imagen...")
        image = pipe(prompt).images[0]
        return image
    except Exception as e:
        print(f"Error generando la imagen: {str(e)}")
        return f"Error generando la imagen: {str(e)}"

async def generate_text(prompt):
    """
    Genera texto basado en un prompt usando un modelo de Transformers.
    """
    try:
        # Cargar el modelo
        model_id = models["text-generation"]
        print("Cargando modelo de Transformers para text-generation...")
        text_generator = pipeline("text-generation", model=model_id)

        # Generar texto
        print("Generando texto...")
        output = text_generator(prompt, max_length=100, num_return_sequences=1)
        return output[0]["generated_text"]
    except Exception as e:
        print(f"Error generando el texto: {str(e)}")
        return f"Error generando el texto: {str(e)}"

# Paso 4: Detección de tarea
def detect_task(prompt):
    """
    Detecta la tarea en base al contenido del prompt.
    """
    if any(keyword in prompt.lower() for keyword in ["imagen", "dibujo", "gráfico"]):
        return "text-to-image"
    else:
        return "text-generation"  # Tarea por defecto

# Paso 5: Procesar el prompt
async def process_prompt(prompt):
    """
    Procesa el prompt y devuelve la salida según la tarea detectada.
    """
    task = detect_task(prompt)

    # Liberar memoria antes de cargar modelos
    torch.cuda.empty_cache()
    gc.collect()

    if task == "text-to-image":
        return await generate_image(prompt)
    elif task == "text-generation":
        return await generate_text(prompt)

# Paso 6: Interfaz con Gradio
async def main_function(prompt):
    """
    Detecta la tarea y devuelve el resultado en la interfaz.
    """
    try:
        print("Iniciando procesamiento...")
        text_task = asyncio.create_task(process_prompt(prompt))
        image_task = asyncio.create_task(process_prompt(prompt))

        # Esperar a que ambas tareas se completen
        image_result = await image_task
        text_result = await text_task

        return image_result, text_result
    except Exception as e:
        print(f"Error procesando tu solicitud: {str(e)}")
        return None, f"Error procesando tu solicitud: {str(e)}"

# Configuración de la interfaz
interface = gr.Interface(
    fn=main_function,
    inputs=gr.Textbox(label="Prompt", placeholder="Describe lo que necesitas: texto o imagen"),
    outputs=[
        gr.Image(label="Resultado (Imagen)", type="pil"),
        gr.Textbox(label="Resultado (Texto)")
    ],
    title="Chat Multi-Modo",
    description="Detecta automáticamente si necesitas texto o imágenes y usa el modelo adecuado."
)

# Lanzar la interfaz
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f360f4b38ff460e06c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
# Paso 1: Instalación de librerías necesarias
!pip install gradio transformers diffusers accelerate safetensors -q

# Paso 2: Importación y configuración
import torch
from transformers import pipeline
from diffusers import StableDiffusionPipeline
import gradio as gr
import gc

# Modelos específicos según la tarea
models = {
    "text-to-image": "runwayml/stable-diffusion-v1-5",  # Texto a imagen
    "text-generation": "gpt2",  # Generación de texto
}

# Paso 3: Funciones para cada tarea
def generate_image(prompt):
    """
    Genera una imagen basada en el texto usando Diffusers.
    """
    try:
        # Verificar disponibilidad de GPU
        if torch.cuda.is_available():
            device = "cuda"
            torch_dtype = torch.float16
            print("Usando GPU para la generación de imagen.")
        else:
            device = "cpu"
            torch_dtype = torch.float32
            print("Advertencia: GPU no disponible. Usando CPU, lo cual puede ser más lento.")

        # Cargar el modelo
        model_id = models["text-to-image"]
        print("Cargando modelo de Diffusers para text-to-image...")
        pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch_dtype)
        pipe = pipe.to(device)

        # Generar imagen
        print("Generando imagen...")
        image = pipe(prompt).images[0]
        return image
    except Exception as e:
        error_msg = f"Error generando la imagen: {str(e)}"
        print(error_msg)
        return error_msg

def generate_text(prompt):
    """
    Genera texto basado en un prompt usando un modelo de Transformers.
    """
    try:
        # Cargar el modelo
        model_id = models["text-generation"]
        print("Cargando modelo de Transformers para text-generation...")
        text_generator = pipeline("text-generation", model=model_id)

        # Generar texto
        print("Generando texto...")
        output = text_generator(prompt, max_length=100, num_return_sequences=1)
        return output[0]["generated_text"]
    except Exception as e:
        error_msg = f"Error generando el texto: {str(e)}"
        print(error_msg)
        return error_msg

# Paso 4: Interfaz principal
def main_function(prompt):
    """
    Detecta las tareas y devuelve los resultados en la interfaz.
    """
    try:
        print("Iniciando procesamiento...")
        # Procesar imagen
        print("Procesando imagen...")
        image_result = generate_image(prompt)

        # Procesar texto
        print("Procesando texto...")
        text_result = generate_text(prompt)

        return image_result, text_result
    except Exception as e:
        error_msg = f"Error procesando tu solicitud: {str(e)}"
        print(error_msg)
        return None, error_msg

# Configuración de la interfaz
interface = gr.Interface(
    fn=main_function,
    inputs=gr.Textbox(label="Prompt", placeholder="Describe lo que necesitas: texto o imagen"),
    outputs=[
        gr.Image(label="Resultado (Imagen)", type="pil"),
        gr.Textbox(label="Resultado (Texto)")
    ],
    title="Chat Multi-Modo",
    description="Detecta automáticamente si necesitas texto o imágenes y usa el modelo adecuado."
)

# Lanzar la interfaz
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dcd662f8dc4bfa021b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
